In [ ]:
from beakerx import *
from beakerx.object import beakerx
from ipywidgets import widgets, interactive
from ipywidgets import Layout
from IPython.display import display
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly
import pandas as pd
import trafero
from datetime import datetime
import warnings
warnings.simplefilter("ignore")

In [ ]:
class ASUPConnector:

    def __init__(self, status_context=None):
        self.name = "asup"
        self.trafero_handle = trafero.Asup()
        self.workspace_id = None
        self.status_context = status_context
        self.trafero_handle._log = self._log

    def grab_status_context(self, element):
        self.status_context = element

    def _log(self, msg, verbose):
        if self.status_context:
            self.status_context.value = msg

    def safe_ingest(self, workspace_id, asup_ids, processes):
        self.workspace_id = workspace_id
        results = None

        if self.status_context:
            self.status_context.value = "Parsing CCMA Files..."

        try:
            results = self.trafero_handle.ingest(
                asup_ids=asup_ids,
                workspace_id=self.workspace_id,
                processes=processes
            )
        except Exception as ex:
            if self.status_context:
                self.status_context.value = "{0}".format(ex)

        return results

    def get_workspace_ids(self):
        results = self.trafero_handle.get_workspace_ids()
        if results.empty or not results['workspace_id'][0]:
            return []

        workspace_ids = list(results['workspace_id'][0])
        workspace_ids.sort(reverse=True)

        return workspace_ids

    def get_view(self, workspace_id):
        self.workspace_id = workspace_id

        if self.status_context:
            self.status_context.value = "Searching..."

        results = self.trafero_handle.get_workspace_ids(prefix=workspace_id)

        if not results['workspace_id'][0]:
            if self.status_context:
                self.status_context.value = "Workspace not found"
            return

        if self.status_context:
            self.status_context.value = "Fetching Data..."

        return CCMAView(self)


class CCMAView:

    def __init__(self, connector):
        self.connector = connector
        self.cluster_name = None
        self.node_names = None
        self.start_time = None
        self.stop_time = None
        self.list_cluster_names = []
        self.list_node_names = []

        if self.connector.name == "trafero":
            self.update_list_cluster_names_files()
        else:
            self.update_list_cluster_names_workspace()

        self.update_list_node_names()

    def get_cluster_names(self):
        return self.list_cluster_names

    def get_node_names(self):
        return self.list_node_names

    def update_list_cluster_names_files(self):
        temp_refernce_object = self.get_appended_files()

        self.list_cluster_names = list(
            temp_refernce_object['cluster_name'].unique())

        if self.list_cluster_names:
            self.cluster_name = self.list_cluster_names[0]

    def update_list_cluster_names_workspace(self):
        temp_refernce_object = self.connector.trafero_handle.get_clusters(
            self.connector.workspace_id)

        self.list_cluster_names = list(
            temp_refernce_object['cluster_name'].unique())

        if self.list_cluster_names:
            self.cluster_name = self.list_cluster_names[0]

    def update_list_node_names(self):
        if self.connector.name == "trafero":
            temp_refernce_object = self.connector.trafero_handle.get_nodes(
                batch_name=self.connector.workspace_id, cluster=self.cluster_name)
        else:
            temp_refernce_object = self.connector.trafero_handle.get_nodes(
                self.connector.workspace_id, self.cluster_name)

        self.list_node_names = list(temp_refernce_object['node_name'].unique())

        if self.list_node_names:
            self.node_names = [self.list_node_names[0]]

    def set_cluster_name(self, cluster_name):
        self.cluster_name = cluster_name

    def set_node_names(self, node_names):
        self.node_names = node_names

    def set_start_time(self, start_time):
        self.start_time = start_time

    def set_stop_time(self, stop_time):
        self.stop_time = stop_time

    def get_object_list(self):
        if self.connector.name == "trafero":
            return list(self.connector.trafero_handle.get_objects(
                cluster=self.cluster_name,
                node=None,
                batch_name=self.connector.workspace_id)['object_name'].unique())
        else:
            return list(self.connector.trafero_handle.get_objects(
                cluster=self.cluster_name,
                node=None,
                workspace_id=self.connector.workspace_id)['object_name'].unique())

    def get_counter_list(self, cm_object):
        if self.connector.name == "trafero":
            return list(self.connector.trafero_handle.get_counters(
                cluster=self.cluster_name,
                node=None,
                object_name=cm_object,
                batch_name=self.connector.workspace_id)['counter_name'].unique())
        else:
            return list(self.connector.trafero_handle.get_counters(
                cluster=self.cluster_name,
                node=None,
                object_name=cm_object,
                workspace_id=self.connector.workspace_id)['counter_name'].unique())

    def get_instance_list(self, cm_object):
        if self.connector.name == "trafero":
            return list(self.connector.trafero_handle.get_instances(
                cluster=self.cluster_name,
                node=None,
                object_name=cm_object,
                batch_name=self.connector.workspace_id)['instance_name'].unique())
        else:
            return list(self.connector.trafero_handle.get_instances(
                cluster=self.cluster_name,
                node=None,
                object_name=cm_object,
                workspace_id=self.connector.workspace_id)['instance_name'].unique())

    def get_appended_files(self, cluster=None, node=None):
        if self.connector.name == "trafero":
            df_out = self.connector.trafero_handle.get_appended_files(
                batch_name=self.connector.workspace_id,
                cluster=cluster,
                node=node,
            )
        else:
            df_out = self.connector.trafero_handle.get_appended_files(
                workspace_id=self.connector.workspace_id,
                cluster=cluster,
                node=node,
            )

        keys = df_out.keys()
        list_not_to_remove = [
            'appended_file_name', 'cluster_name', 'time_from',
            'time_to', 'node_name',
        ]

        drop_list = list(set(keys).difference(set(list_not_to_remove)))

        df_out = df_out.drop(drop_list, axis=1)
        df_out = df_out.drop(df_out[df_out['time_from'] == 0].index)
        df_out = df_out.drop(df_out[df_out['time_to'] == 0].index)

        return df_out

    def get_time_range(self):
        df_out = self.get_appended_files()

        min_time = list(df_out['time_from'])
        max_time = list(df_out['time_to'])
        min_time = min(min_time)
        max_time = max(max_time)

        return (min_time, max_time)

    def get_values(self, counter_list, time_from=None, time_to=None, node=None,
                   instance_name=None, instance_name_regex=False, min_value=None,
                   max_value=None):
        df_out = pd.DataFrame()

        cm_objects = list(set([x.split(':')[0] for x in counter_list]))
        for cm_object in cm_objects:
            current_object = [x for x in counter_list if cm_object in x]
            current_counters = [x.split(':')[1] for x in current_object]

            if self.connector.name == "trafero":
                cm_object_values = self.connector.trafero_handle.get_values(
                    cluster=self.cluster_name,
                    node=node,
                    batch_name=self.connector.workspace_id,
                    object_name=cm_object,
                    counter_names=current_counters,
                    time_from=time_from,
                    time_to=time_to,
                    instance_name=instance_name,
                    instance_name_regex=instance_name_regex,
                    min_value=min_value,
                    max_value=max_value,
                    best_effort=True)
            else:
                cm_object_values = self.connector.trafero_handle.get_values(
                    cluster=self.cluster_name,
                    node=node,
                    workspace_id=self.connector.workspace_id,
                    object_name=cm_object,
                    counter_names=current_counters,
                    time_from=time_from,
                    time_to=time_to,
                    instance_name=instance_name,
                    instance_name_regex=instance_name_regex,
                    min_value=min_value,
                    max_value=max_value,
                    best_effort=True)

            if node is not None:
                cm_object_values = cm_object_values[
                    cm_object_values['node_name'] == node]

            for current_counter in current_counters:
                current_values = cm_object_values[
                    cm_object_values['counter_name'] == current_counter]

                keys = current_values.keys()
                list_not_to_remove = [
                    'timestamp', 'x_label', 'instance_name',
                    'counter_value', 'counter_name', 'cluster_name',
                    'object_name', 'node_name',
                ]

                drop_list = list(set(keys).difference(set(list_not_to_remove)))

                current_values = current_values.drop(drop_list, axis=1)
                current_values['object_counter_name'] = current_values['object_name'] + ':' + current_values[
                    'counter_name']

                # Remove excessive values
                if 'counter_value' in current_values.keys():
                    if max_value is not None and self.is_number(max_value):
                        current_values = current_values.drop(
                            current_values[current_values['counter_value'] > float(max_value)].index)

                    if min_value is not None and self.is_number(min_value):
                        current_values = current_values.drop(
                            current_values[current_values['counter_value'] < float(min_value)].index)

                x_labels = []
                if 'x_label' in current_values.keys():
                    x_labels = current_values['x_label'].unique()
                    # remove empty string
                    x_labels = [x for x in x_labels if x]

                if len(x_labels) > 0:
                    for x_label in x_labels:
                        x_labels_df = current_values[current_values['x_label'] == x_label]
                        x_labels_df['object_counter_name'] = x_labels_df['object_name'] + ':' + x_labels_df[
                            'counter_name'] + ":" + x_label
                        df_out = df_out.append(x_labels_df, sort=True)
                else:
                    df_out = df_out.append(current_values, sort=True)

        df_out = pd.pivot_table(
            df_out,
            index=[
                'timestamp',
                'cluster_name',
                'node_name',
                'instance_name',
                'x_label',
                'object_name',
                'counter_name',
            ],
            columns='object_counter_name',
            values='counter_value',
            aggfunc='first').reset_index()

        df_out = df_out.drop(
            ['x_label', 'object_name', 'counter_name'], axis=1)
        df_out = df_out.groupby(
            ['timestamp', 'cluster_name', 'node_name', 'instance_name']).sum()

        index_timestamp = [x[0] for x in df_out.index.tolist()]
        index_cluster_name = [x[1] for x in df_out.index.tolist()]
        index_node_name = [x[2] for x in df_out.index.tolist()]
        index_instance_name = [x[3] for x in df_out.index.tolist()]

        df_out.insert(0, 'timestamp', index_timestamp)
        df_out.insert(1, 'plot_timestamp', index_timestamp)
        df_out.insert(2, 'cluster_name', index_cluster_name)
        df_out.insert(3, 'node_name', index_node_name)
        df_out.insert(4, 'instance_name', index_instance_name)
        # beakerx 1.4.0 removes the index column before downloading to csv
        df_out.reset_index(drop=True, inplace=True)

        return df_out.fillna(0)

    def is_number(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

class TimeBlock:

    def __init__(self, cluster_name, node_name, location, start_time, end_time):
        self.cluster_name = cluster_name
        self.node_name = node_name
        self.location = location
        self.start_time = start_time
        self.end_time = end_time


class FilesViewer:

    def parse_appended_file_type(self, path_value):
        file_location = path_value.rsplit("/", maxsplit=1)

        if file_location:
            return file_location[0]
        else:
            return ""

    def create_show_all_ingested(self, df_info):
        columns_af = ['Cluster Name', 'Node Name',
                      'Type', 'Start Time', 'End Time']

        frame_af = pd.DataFrame(data=None, columns=columns_af)

        frame_af['Cluster Name'] = df_info['cluster_name']
        frame_af['Node Name'] = df_info['node_name']
        frame_af['Type'] = [self.parse_appended_file_type(
            x) for x in df_info['appended_file_name']]
        frame_af['Start Time'] = [
            datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f') for x in df_info['time_from']]
        frame_af['End Time'] = [
            datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f') for x in df_info['time_to']]

        return frame_af

    def create_show_gaps_view(self, df_info, threshold):
        columns_af_gap = ['Cluster Name', 'Node Name',
                          'Type', 'Gap Start Time', 'Gap End Time']

        frame_af_gap = pd.DataFrame(data=None, columns=columns_af_gap)

        if self.is_number(threshold):
            threshold = float(threshold)

        gaps = self.find_gaps(df_info, threshold)

        for i in range(len(gaps)):
            frame_af_gap.set_value(i, 'Cluster Name', gaps[i].cluster_name)
            frame_af_gap.set_value(i, 'Node Name', gaps[i].node_name)
            frame_af_gap.set_value(i, 'Type', gaps[i].location)
            frame_af_gap.set_value(i, 'Gap Start Time',
                                   datetime.fromtimestamp(gaps[i].start_time).strftime('%Y-%m-%d %H:%M:%S.%f'))
            frame_af_gap.set_value(i, 'Gap End Time',
                                   datetime.fromtimestamp(gaps[i].end_time).strftime('%Y-%m-%d %H:%M:%S.%f'))

        return frame_af_gap

    def find_gaps(self, df_data, threshold):
        df_data_sorted = df_data.sort_values(by=['time_to', 'time_from'])

        appended_files = list(df_data_sorted['appended_file_name'])
        cluster_names = list(df_data_sorted['cluster_name'])
        node_names = list(df_data_sorted['node_name'])
        start_times = list(df_data_sorted['time_from'])
        end_times = list(df_data_sorted['time_to'])

        timelines = {}
        gaps = []

        # Filtering blocks
        for i in range(len(start_times)):
            file_location = self.parse_appended_file_type(appended_files[i])

            block_info = TimeBlock(
                cluster_name=cluster_names[i],
                node_name=node_names[i],
                location=file_location,
                start_time=start_times[i],
                end_time=end_times[i])

            timeline_key = "{0}:{1}:{2}".format(
                cluster_names[i], node_names[i], file_location)

            if timeline_key not in timelines:
                temp_list = list()
                temp_list.append(block_info)

                timelines[timeline_key] = temp_list
            else:
                timelines[timeline_key].append(block_info)

        for key in timelines:
            curr_timeline = timelines[key]

            for i in range(len(curr_timeline)):
                if i + 1 < len(curr_timeline):
                    time_result = abs(
                        curr_timeline[i+1].start_time - curr_timeline[i].end_time) / 1000 / 60

                    if time_result > threshold:
                        gaps.append(TimeBlock(
                            cluster_name=curr_timeline[i].cluster_name,
                            node_name=curr_timeline[i].node_name,
                            location=curr_timeline[i].location,
                            start_time=curr_timeline[i].end_time / 1000.0,
                            end_time=curr_timeline[i+1].start_time / 1000.0))

        return gaps

    def is_number(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

        

class ExplorerFormEvents:

    def __init__(self, form):
        self.form = form

    def clear_query_widgets(self):
        try:  # avoid some corner cases with the try block
            # reset widgets
            self.form.query_object_list.disabled = True
            self.form.query_cluster_name.disabled = True
            self.form.query_node_names.disabled = True
            self.form.query_counter_selector.disabled = True
            self.form.query_filter_time.disabled = True
            self.form.query_display_counters_button.disabled = True
        except Exception as ex:
            pass

    def clear_view_files_widgets(self):
        try:  # avoid some corner cases with the try block
            # reset widgets
            self.form.view_files_cluster_name.disabled = True
            self.form.view_files_node_names.disabled = True
            self.form.view_files_display_button.disabled = True
        except Exception as ex:
            pass

    def update_time_filter_display(self):
        if len(self.form.query_filter_time.value) == 2:
            start_time = self.form.query_filter_time.value[0]
            end_time = self.form.query_filter_time.value[1]

            start_time_conv = datetime.fromtimestamp(start_time / 1000.0)
            end_time_conv = datetime.fromtimestamp(end_time / 1000.0)

            self.form.query_filter_time_start.value = start_time_conv.strftime(
                '%Y-%m-%d %H:%M:%S.%f')
            self.form.query_filter_time_end.value = end_time_conv.strftime(
                '%Y-%m-%d %H:%M:%S.%f')

    def build_counter_views(self, counter_list, node_name, node_result, build_tables, build_plots):
        if build_tables:
            self.form.status_query.value = "Building table for {0}...".format(
                node_name)
            self.form.tab_view_labels.append("Table: {0}".format(node_name))
            self.form.tab_view_childrens.append(
                self.form.create_table_view(node_result))

        if build_plots:
            recv_counter_list = []

            for column_name in node_result.keys():
                for counter_name in counter_list:
                    if column_name.startswith(counter_name):
                        recv_counter_list.append(column_name)

            for counter_name in recv_counter_list:
                self.form.status_query.value = "Building plot for {0}: {1}...".format(
                    node_name, counter_name)
                self.form.tab_view_labels.append(
                    "Plot: {0}: {1}".format(node_name, counter_name))
                plot_fig = self.form.create_plot_view(
                    node_result, counter_name)
                self.form.tab_view_childrens.append(plot_fig)

    def ingest_button_on_click(self, change):
        self.form.connector.grab_status_context(self.form.status_ccma)

        try:
            if self.form.ingest_workspace_id.value == '':
                self.form.status_ccma.value = 'Error: No {0}'.format(
                    self.form.profile.workspace_id)
                return

            self.form.ingest_button.disabled = True

            workspace_id = self.form.ingest_workspace_id.value
            record_ids = self.form.ingest_record_ids.value
            record_ids = record_ids.replace(" ", "")
            record_ids = record_ids.split(',')

            if self.form.ingest_checkbox.value == True:
                ingest_results = self.form.connector.safe_ingest(workspace_id, record_ids, processes=None)  
            else:
                ingest_results = self.form.connector.safe_ingest(workspace_id, record_ids, processes=['kernel'])

            self.form.select_tab_view.disabled = True
            self.form.tab_view_childrens = [self.form.create_table_view(ingest_results)]
            self.form.tab_view_labels = ["Ingest Results"]
            self.form.select_tab_view.options = self.form.tab_view_labels
            self.form.select_tab_view.disabled = False
            self.form.interactive_tab_view.update()
        except Exception as ex:
            self.form.ingest_button.disabled = False
            self.form.status_ccma.value = "Error: {0} could not be ingested: {1}".format(
                self.form.profile.workspace_plural, ex)
            return

        self.form.ingest_button.disabled = False
        self.form.status_ccma.value = 'Finished'

    # View Ingested Files
    def view_files_retrieve_button_on_click(self, change):
        self.form.connector.grab_status_context(self.form.status_view_files)

        try:
            if self.form.view_files_workspace_id.value == '':
                self.form.status_view_files.value = "Error: {0} could not be retrieved".format(
                    self.form.profile.workspace)
                return

            self.form.view_files_retrieve_button.disabled = True
            self.clear_view_files_widgets()

            self.form.ccma_view = self.form.connector.get_view(
                self.form.view_files_workspace_id.value)

            self.form.status_view_files.value = "Fetching Clusters..."
            self.form.view_files_cluster_name.disabled = False
            self.form.view_files_cluster_name.options = self.form.ccma_view.get_cluster_names()

            self.form.status_view_files.value = "Fetching Nodes..."
            self.form.view_files_node_names.disabled = False
            self.form.view_files_node_names.options = self.form.ccma_view.get_node_names()

            if self.form.view_files_node_names.options:
                self.form.view_files_node_names.value = [
                    self.form.view_files_node_names.options[0]]
        except Exception as ex:
            self.form.status_view_files.value = "{0}".format(ex)
            self.form.view_files_retrieve_button.disabled = False
            return

        self.form.view_files_retrieve_button.disabled = False
        self.form.view_files_display_button.disabled = False
        self.form.status_view_files.value = "Select options and click 'Display'"

    def view_files_refresh_workspace_ids_button_on_click(self, change):
        self.form.connector.grab_status_context(self.form.status_view_files)

        self.form.status_view_files.value = (
            "Fetching {0}...".format(self.form.profile.workspace_id_plural))
        self.form.view_files_workspace_id.options = self.form.connector.get_workspace_ids()
        self.form.status_view_files.value = "Select a {0} and click on 'Fetch {1}'".format(
            self.form.profile.workspace_id, self.form.profile.workspace)

    def view_files_display_button_on_click(self, change):
        self.form.view_files_display_button.disabled = True
        node_search = self.form.view_files_node_names.value
        node_search_options = self.form.view_files_node_names.options
        use_filter = self.form.view_files_filter_selector.value

        if use_filter and not node_search:
            self.form.view_files_display_button.disabled = False
            self.form.status_view_files.value = "No nodes selected"
            return

        self.form.select_tab_view.disabled = True

        try:
            self.form.tab_view_childrens = []
            self.form.tab_view_labels = []

            if use_filter:
                node_results = []

                filter_cluster_name = self.form.view_files_cluster_name.value

                if len(node_search) == len(node_search_options):
                    info_af = self.form.ccma_view.get_appended_files(
                        filter_cluster_name, None)
                else:
                    for x in node_search:
                        node_results.append(
                            self.form.ccma_view.get_appended_files(filter_cluster_name, x))

                    info_af = pd.concat(node_results)
            else:
                info_af = self.form.ccma_view.get_appended_files()

            self.form.tab_view_labels.append("View Ingested Files")
            view_files = self.form.create_table_view(
                self.form.time_viewer.create_show_all_ingested(info_af))
            self.form.tab_view_childrens.append(view_files)

            self.form.tab_view_labels.append("View Gaps")
            gap_threshold = self.form.view_files_gap_threshold_value.value
            view_gaps = self.form.create_table_view(
                self.form.time_viewer.create_show_gaps_view(info_af, gap_threshold))
            self.form.tab_view_childrens.append(view_gaps)

            self.form.select_tab_view.options = self.form.tab_view_labels
            self.form.interactive_tab_view.update()
        except Exception as ex:
            self.form.status_view_files.value = "{0}".format(ex)
            return

        self.form.view_files_display_button.disabled = False
        self.form.select_tab_view.disabled = False

    def query_fetch_workspace_information(self, workspace_id):
        self.form.connector.grab_status_context(self.form.status_query)

        try:
            if workspace_id == '':
                self.form.status_query.value = "Error: {0} could not be retrieved".format(
                    self.form.profile.workspace)
                return

            self.form.query_retrieve_button.disabled = True
            self.clear_query_widgets()

            self.form.ccma_view = self.form.connector.get_view(workspace_id)

            self.form.status_query.value = "Fetching Clusters..."
            self.form.query_cluster_name.disabled = False
            self.form.query_cluster_name.options = self.form.ccma_view.get_cluster_names()

            self.form.status_query.value = "Fetching Nodes..."
            self.form.query_node_names.disabled = False
            self.form.query_node_names.options = self.form.ccma_view.get_node_names()

            if self.form.query_node_names.options:
                self.form.query_node_names.value = [
                    self.form.query_node_names.options[0]]

            self.form.status_query.value = "Fetching Objects..."
            try:
                self.form.query_object_list.disabled = False
                self.form.query_object_list.options = self.form.ccma_view.get_object_list()
            except Exception as ex:
                self.form.status_query.value = 'Error: Objects could not be retrieved: {0}'.format(
                    ex)
                return
        except Exception as ex:
            self.form.query_retrieve_button.disabled = False
            self.clear_query_widgets()
            self.form.status_query.value = 'Error: {0} could not be retrieved: {1}'.format(
                self.form.profile.workspace, ex)
            return

        self.form.query_retrieve_button.disabled = False
        self.form.query_filter_time.disabled = False
        self.form.query_filter_instance_name_regex.disabled = False
        self.form.query_filter_instance_name.disabled = False
        self.form.query_counter_selector.disabled = False
        self.form.query_display_counters_button.disabled = False

        try:
            min_time, max_time = self.form.ccma_view.get_time_range()

            if max_time < self.form.query_filter_time.min:
                self.form.query_filter_time.min = min_time
                self.form.query_filter_time.max = max_time
            else:
                self.form.query_filter_time.max = max_time
                self.form.query_filter_time.min = min_time
            self.form.query_filter_time.value = [min_time, max_time]

            self.update_time_filter_display()
        except Exception as ex:
            self.form.status_query.value = 'Error: {0}'.format(ex)
            return

        self.form.status_query.value = "Select counters and click 'Fetch Counter Data'"

    # Query
    def query_retrieve_button_on_click(self, change):
        self.query_fetch_workspace_information(
            self.form.query_workspace_id.value)

    def query_refresh_workspace_ids_button_on_click(self, change):
        self.form.connector.grab_status_context(self.form.status_query)

        self.form.status_query.value = ("Fetching {0}...".format(
            self.form.profile.workspace_id_plural))
        self.form.query_workspace_id.options = self.form.connector.get_workspace_ids()
        self.form.status_query.value = "Select a workspace id and click on 'Fetch Workspace'"

    def query_cluster_name_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.form.query_cluster_name.value == '':
                return

            self.form.status_query.value = "Fetching Nodes..."
            try:
                self.form.ccma_view.set_cluster_name(
                    self.form.query_cluster_name.value)
                self.form.ccma_view.update_list_node_names()
                self.form.query_node_names.options = self.form.ccma_view.get_node_names()

                if self.form.query_node_names.options:
                    self.form.query_node_names.value = [
                        self.form.query_node_names.options[0]]
            except Exception as ex:
                self.clear_query_widgets()
                self.form.status_query.value = 'Error: Nodes could not be retrieved: {0}'.format(
                    ex)

    def query_node_names_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if not self.form.query_node_names.value:
                return

            self.form.ccma_view.set_node_names(
                self.form.query_node_names.value)

    def query_object_list_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.form.query_object_list.value == '':
                return

            self.form.query_counter_selector.disabled = True
            self.form.query_filter_instance_name.disabled = True

            counter_list = sorted(self.form.ccma_view.get_counter_list(
                self.form.query_object_list.value))
            instance_list = sorted(self.form.ccma_view.get_instance_list(
                self.form.query_object_list.value))

            self.form.status_query.value = "Fetching Counters..."
            self.form.query_counter_selector.disabled = False
            self.form.query_counter_selector.options = counter_list
            self.form.query_filter_instance_name.disabled = False
            self.form.query_filter_instance_name.options = instance_list
            self.form.query_instance_name_selector.value = "None"
            self.form.status_query.value = "Select counters and click 'Fetch Counter Data'"

    def query_time_filter_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.form.query_filter_time.value:
                self.update_time_filter_display()
            return

    def query_instance_name_filter_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.form.query_instance_name_selector.value = "Instance Name"

    def query_instance_name_regex_filter_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.form.query_instance_name_selector.value = "Instance Name Regex"

    def query_display_counters_button_on_click(self, change):
        self.form.query_display_counters_button.disabled = True

        if not self.form.query_node_names.value:
            self.form.query_display_counters_button.disabled = False
            self.form.status_query.value = "No nodes selected"
            return

        counter_list = [self.form.query_object_list.value + ':' + x for x in
                        self.form.query_counter_selector.value]

        if not counter_list:
            self.form.query_display_counters_button.disabled = False
            self.form.status_query.value = "No counters selected"
            return

        filter_time_start = None
        filter_time_end = None

        if len(self.form.query_filter_time.value) >= 2:
            filter_time_start = self.form.query_filter_time.value[0]
            filter_time_end = self.form.query_filter_time.value[1]

        filter_instance_name = None
        filter_instance_name_regex = False

        if self.form.query_instance_name_selector.value == "Instance Name":
            if len(self.form.query_filter_instance_name.value) > 0:
                filter_instance_name = self.form.query_filter_instance_name.value
        elif self.form.query_instance_name_selector.value == "Instance Name Regex":
            if len(self.form.query_filter_instance_name_regex.value) > 0:
                filter_instance_name = self.form.query_filter_instance_name_regex.value
                filter_instance_name_regex = True

        filter_min_value = None
        filter_max_value = None

        if len(self.form.query_filter_min_value.value) > 0:
            filter_min_value = self.form.query_filter_min_value.value

        if len(self.form.query_filter_max_value.value) > 0:
            filter_max_value = self.form.query_filter_max_value.value

        build_tables = self.form.query_make_table.value
        build_plots = self.form.query_make_plot.value
        merge_nodes = self.form.query_merge_results.value

        if not build_tables and not build_plots:
            self.form.query_display_counters_button.disabled = False
            self.form.status_query.value = "Must select a table or plot"
            return

        self.form.status_query.value = "Retrieving counters..."

        node_search = self.form.query_node_names.value
        node_results = []

        full_cluster = len(self.form.query_node_names.value) == len(
            self.form.query_node_names.options)

        try:
            if full_cluster:
                node_results.append(self.form.ccma_view.get_values(
                    counter_list=counter_list,
                    time_from=filter_time_start,
                    time_to=filter_time_end,
                    node=None,
                    instance_name=filter_instance_name,
                    instance_name_regex=filter_instance_name_regex,
                    min_value=filter_min_value,
                    max_value=filter_max_value))
            else:
                for x in node_search:
                    node_results.append(self.form.ccma_view.get_values(
                        counter_list=counter_list,
                        time_from=filter_time_start,
                        time_to=filter_time_end,
                        node=x,
                        instance_name=filter_instance_name,
                        instance_name_regex=filter_instance_name_regex,
                        min_value=filter_min_value,
                        max_value=filter_max_value))
        except Exception as ex:
            self.form.status_query.value = "{0}".format(ex)
            self.form.query_display_counters_button.disabled = False
            return

        self.form.select_tab_view.disabled = True
        self.form.tab_view_childrens = []
        self.form.tab_view_labels = []

        try:
            if merge_nodes or full_cluster:
                merged_results = pd.concat(node_results)

                if full_cluster:
                    self.build_counter_views(
                        counter_list, "Cluster", merged_results, build_tables, build_plots)
                else:
                    self.build_counter_views(
                        counter_list, "All Selected Nodes", merged_results, build_tables, build_plots)
            else:
                for x in node_results:
                    if not x['node_name'].empty:
                        node_name = x['node_name'][0]
                    else:
                        node_name = ''

                    self.build_counter_views(counter_list, node_name, x,
                                             build_tables, build_plots)
        except Exception as ex:
            self.form.status_query.value = "{0}".format(ex)
            self.form.query_display_counters_button.disabled = False
            return

        self.form.select_tab_view.options = self.form.tab_view_labels

        self.form.interactive_tab_view.update()
        self.form.status_query.value = ''

        self.form.query_display_counters_button.disabled = False
        self.form.select_tab_view.disabled = False

    def notebook_url_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.form.notebook_url.value == '':
                return

            self.form.parse_notebook_url(self.form.notebook_url.value)

    def select_tab_view_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.form.select_tab_view.value == '':
                return

        self.form.interactive_tab_view.update()

    def ingest_checkbox_on_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value' and change['new']:
            self.form.ingest_checkbox_label.layout.visibility="visible"
        else:
            self.form.ingest_checkbox_label.layout.visibility="hidden"
        

class ExplorerProfile:

    def __init__(self, workspace, workspace_plural, workspace_id, workspace_id_plural, record_id, record_id_plural):
        self.workspace = workspace
        self.workspace_plural = workspace_plural
        self.workspace_id = workspace_id
        self.workspace_id_plural = workspace_id_plural
        self.record_id = record_id
        self.record_id_plural = record_id_plural


class ExplorerForm:

    def __init__(self, connector_type, profile, service_url=None):
        self.profile = profile
        self.connector_type = connector_type

        if connector_type == 'perfant':
            self.connector = PerfantConnector()
        elif connector_type == 'asup':
            self.connector = ASUPConnector()
        elif connector_type == 'trafero':
            self.connector = TraferoConnector(trafero_url=service_url)
        else:
            self.connector = None

        self.time_viewer = FilesViewer()
        self.events = ExplorerFormEvents(self)
        self.ccma_view = None
        self.counter_list = None
        self.tab_view_childrens = []
        self.tab_view_labels = []

    def parse_notebook_url(self, url):
        from urllib import parse

        try:
            field_list = parse.parse_qs(parse.urlsplit(url).query)

            if field_list:
                workspace_id = ""

                if self.connector.name == "trafero":
                    if "batch_name" in field_list and field_list["batch_name"]:
                        workspace_id = field_list["batch_name"][0]
                elif self.connector.name == "perfant":
                    if "run_id" in field_list and field_list["run_id"]:
                        workspace_id = "perfant_" + field_list["run_id"][0]
                elif self.connector.name == "asup":
                    if "asup_id" in field_list and field_list["asup_id"]:
                        workspace_id = field_list["asup_id"][0]

                if workspace_id:
                    self.query_workspace_id.value = workspace_id
                    self.view_files_workspace_id.value = workspace_id
                    self.events.query_fetch_workspace_information(
                        workspace_id=workspace_id)
        except:
            pass

    def build_entry_form_ingest(self, form_item_layout):
        self.ingest_workspace_id = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=False,
        )
        self.ingest_workspace_id_hbox = widgets.HBox([
            widgets.Label("{0}:".format(self.profile.workspace_id)),
            self.ingest_workspace_id,
        ], layout=form_item_layout)

        self.ingest_record_ids = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=False,
        )
        self.ingest_record_ids_hbox = widgets.VBox([
            widgets.Label("{0}:".format(self.profile.record_id_plural)),
            self.ingest_record_ids,
        ], layout=form_item_layout)

        self.ingest_button = widgets.Button(
            description='Ingest CCMA',
            disabled=False,
            button_style='',
            tooltip='Ingest performance data',
        )

        self.ingest_checkbox = widgets.Checkbox(
            value=False,
            description='Ingest archives for all processes',
            indent=False
        )
        
        self.ingest_checkbox_label = widgets.HTML(
            value = f"<b><font color='gray'><p/> Default is ONTAP kernel only. Ingesting archives for <u>ALL</u> "
            f"processes will greatly increase ingest times.</p></b>"
        )
        
        self.ingest_checkbox_vbox = widgets.VBox([
            self.ingest_checkbox, self.ingest_checkbox_label
        ])
        self.ingest_button_hbox = widgets.HBox([
            widgets.Label(""), self.ingest_button, self.ingest_checkbox_vbox
        ])

    def build_entry_form_query(self, form_item_layout):
        self.query_workspace_id = widgets.Combobox(
            options=[''],
            value='',
            ensure_option=True,
            disabled=False,
        )
        self.query_workspace_id_hbox = widgets.HBox([
            widgets.Label("{0}:".format(self.profile.workspace_id)),
            self.query_workspace_id,
        ], layout=form_item_layout)
        self.query_workspace_id.options = self.connector.get_workspace_ids()

        self.query_cluster_name = widgets.Dropdown(
            options=[''],
            value='',
            disabled=True,
        )
        self.query_cluster_name_hbox = widgets.HBox([
            widgets.Label("Cluster:"), self.query_cluster_name,
        ], layout=form_item_layout)

        self.query_node_names = widgets.SelectMultiple(
            options=[],
            value=[],
            description='',
            disabled=True,
            rows=4,
        )
        self.query_node_names_hbox = widgets.HBox([
            widgets.Label("Node(s):"), self.query_node_names,
        ], layout=form_item_layout)

        self.query_counter_selector = widgets.SelectMultiple(
            options=[],
            value=[],
            description='',
            disabled=True,
            rows=6,
        )
        self.query_counter_selector_hbox = widgets.HBox([
            widgets.Label("Counter(s):"), self.query_counter_selector,
        ], layout=form_item_layout)

        self.query_object_list = widgets.Dropdown(
            options=[''],
            value='',
            disabled=True,
        )
        self.query_object_list_hbox = widgets.HBox([
            widgets.Label("Object Name:"), self.query_object_list,
        ], layout=form_item_layout)

        self.query_filter_time_start = widgets.Label(
            value="1970-01-01 00:00:00",
            placeholder="",
            description="",
            style={'font_weight': 'bold'},
        )
        self.query_filter_time_start_hbox = widgets.HBox([
            widgets.Label("Start:"), self.query_filter_time_start,
        ], layout=form_item_layout)

        self.query_filter_time_end = widgets.Label(
            value="1970-01-01 00:00:00",
            placeholder="",
            description="",
            style={'font_weight': 'bold'},
        )
        self.query_filter_time_end_hbox = widgets.HBox([
            widgets.Label("End:"), self.query_filter_time_end,
        ], layout=form_item_layout)

        self.query_filter_time = widgets.FloatRangeSlider(
            value=[0, 1],
            min=0,
            max=1,
            step=0.1,
            disabled=True,
            readout=False,
            continuous_update=False,
            orientation='horizontal',
        )
        self.query_filter_time_hbox = widgets.HBox([
            widgets.Label("Time Filter:"), self.query_filter_time,
        ], layout=form_item_layout)

        self.query_instance_name_selector = widgets.Dropdown(
            options=['None', 'Instance Name', 'Instance Name Regex'],
            value='None',
            description='',
            disabled=False,
        )
        self.query_instance_name_selector_hbox = widgets.HBox([
            widgets.Label("Filter By:"), self.query_instance_name_selector,
        ], layout=form_item_layout)

        self.query_filter_instance_name = widgets.Dropdown(
            options=[''],
            value='',
            disabled=True,
        )
        self.query_filter_instance_name_hbox = widgets.HBox([
            widgets.Label("Instance Name:"), self.query_filter_instance_name,
        ], layout=form_item_layout)

        self.query_filter_instance_name_regex = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=True,
        )
        self.query_filter_instance_name_regex_hbox = widgets.HBox([
            widgets.Label("Instance Name Regex:"),
            self.query_filter_instance_name_regex,
        ], layout=form_item_layout)

        self.query_filter_min_value = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=False,
        )
        self.query_filter_min_value_hbox = widgets.HBox([
            widgets.Label("Min Value:"), self.query_filter_min_value,
        ], layout=form_item_layout)

        self.query_filter_max_value = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=False,
        )
        self.query_filter_max_value_hbox = widgets.HBox([
            widgets.Label("Max Value:"), self.query_filter_max_value,
        ], layout=form_item_layout)

        self.query_make_table = widgets.Checkbox(
            value=True,
            description='Make Table',
            disabled=False,
        )

        self.query_make_plot = widgets.Checkbox(
            value=False,
            description='Make Plots',
            disabled=False,
        )

        self.query_merge_results = widgets.Checkbox(
            value=False,
            description='Merge Results',
            disabled=False,
        )

        self.query_retrieve_button = widgets.Button(
            description="Fetch {0}".format(self.profile.workspace),
            disabled=False,
            button_style='',
            tooltip="Fetch {0}:".format(self.profile.workspace),
        )
        self.query_retrieve_button_hbox = widgets.HBox([
            widgets.Label(""), self.query_retrieve_button,
        ])

        self.query_refresh_workspace_ids_button = widgets.Button(
            description="Refresh " + self.profile.workspace_id_plural,
            disabled=False,
            button_style='',
            tooltip="Refresh " + self.profile.workspace_id_plural,
        )
        self.query_refresh_workspace_ids_button_hbox = widgets.HBox([
            widgets.Label(""), self.query_refresh_workspace_ids_button,
        ])

        self.query_display_counters_button = widgets.Button(
            description='Fetch Counter Data',
            disabled=True,
            button_style='',
            tooltip='Fetch Counter Data',
        )
        self.query_display_counters_button_hbox = widgets.HBox([
            widgets.Label(""), self.query_display_counters_button,
        ])

    def build_entry_form_view_files(self, form_item_layout):
        self.view_files_workspace_id = widgets.Combobox(
            options=[''],
            value='',
            disabled=False,
            ensure_option=True,
        )
        self.view_files_workspace_id_hbox = widgets.HBox([
            widgets.Label("{0}:".format(self.profile.workspace_id)),
            self.view_files_workspace_id,
        ], layout=form_item_layout)
        self.view_files_workspace_id.options = self.connector.get_workspace_ids()

        self.view_files_filter_selector = widgets.Checkbox(
            value=False,
            description='Use Cluster and Node',
            disabled=False,
        )
        self.view_files_filter_selector_hbox = widgets.HBox([
            widgets.Label(
                "Filter By:"), self.view_files_filter_selector, widgets.Label(""),
        ], layout=form_item_layout)

        self.view_files_cluster_name = widgets.Dropdown(
            options=[''],
            value='',
            disabled=True,
        )
        self.view_files_cluster_name_hbox = widgets.HBox([
            widgets.Label("Cluster:"), self.view_files_cluster_name,
        ], layout=form_item_layout)

        self.view_files_node_names = widgets.SelectMultiple(
            options=[],
            value=[],
            description='',
            disabled=True,
            rows=4,
        )
        self.view_files_node_names_hbox = widgets.HBox([
            widgets.Label("Node(s):"), self.view_files_node_names,
        ], layout=form_item_layout)

        self.view_files_gap_threshold_value = widgets.Text(
            value="15",
            placeholder='',
            description="",
            disabled=False,
        )
        self.view_files_gap_threshold_value_hbox = widgets.HBox([
            widgets.Label(
                "Gap Threshold (Minutes):"), self.view_files_gap_threshold_value,
        ], layout=form_item_layout)

        self.view_files_retrieve_button = widgets.Button(
            description="Fetch {0}".format(self.profile.workspace),
            disabled=False,
            button_style='',
            tooltip="Fetch {0}:".format(self.profile.workspace),
        )
        self.view_files_retrieve_button_hbox = widgets.HBox([
            widgets.Label(""), self.view_files_retrieve_button,
        ])

        self.view_files_refresh_workspace_ids_button = widgets.Button(
            description="Refresh " + self.profile.workspace_id_plural,
            disabled=False,
            button_style='',
            tooltip="Refresh " + self.profile.workspace_id_plural,
        )
        self.view_files_refresh_workspace_ids_button_hbox = widgets.HBox([
            widgets.Label(""), self.view_files_refresh_workspace_ids_button,
        ])

        self.view_files_display_button = widgets.Button(
            description='Display',
            disabled=True,
            button_style='',
            tooltip='Display',
        )
        self.view_files_display_button_hbox = widgets.HBox([
            widgets.Label(""), self.view_files_display_button,
        ])

    def build_status_form(self, form_item_layout):
        self.notebook_url = widgets.Text(
            value="",
            placeholder='',
            description="",
            disabled=False,
            tooltip='notebook_url',
        )

        self.status_ccma = widgets.Label(
            value="",
            placeholder="",
            description="",
            disabled=True,
            style={'font_weight': 'bold'},
        )
        self.status_ccma_hbox = widgets.HBox([
            widgets.Label(""), self.status_ccma,
        ], layout=form_item_layout)

        self.status_query = widgets.Label(
            value="Select a {0} and click on 'Fetch {1}'".format(
                self.profile.workspace_id, self.profile.workspace),
            placeholder="",
            description="",
            disabled=True,
            style={'font_weight': 'bold'},
        )
        self.status_query_hbox = widgets.HBox([
            widgets.Label(""), self.status_query,
        ], layout=form_item_layout)

        self.status_view_files = widgets.Label(
            value="Select a {0} and click on 'Fetch {1}'".format(
                self.profile.workspace_id, self.profile.workspace),
            placeholder="",
            description="",
            disabled=True,
            style={'font_weight': 'bold'},
        )
        self.status_view_files_hbox = widgets.HBox([
            widgets.Label(""), self.status_view_files,
        ], layout=form_item_layout)

    def build_form(self):
        form_item_layout = Layout(
            display='flex',
            flex_flow='row',
            justify_content='space-between',
        )

        self.select_tab_view = widgets.Dropdown(
            options=[''],
            value='',
            disabled=False,
        )

        self.build_entry_form_ingest(form_item_layout)
        self.build_entry_form_view_files(form_item_layout)
        self.build_entry_form_query(form_item_layout)
        self.build_status_form(form_item_layout)

        self.ingest_section = [
            self.ingest_workspace_id_hbox,
            self.ingest_record_ids_hbox,
            self.ingest_button_hbox,
            self.status_ccma_hbox,
        ]

        self.query_section_1 = [
            self.query_workspace_id_hbox,
            widgets.HBox([
                self.query_refresh_workspace_ids_button_hbox,
                self.query_retrieve_button_hbox,
            ], layout=form_item_layout),
            self.query_cluster_name_hbox,
            self.query_node_names_hbox,
            self.query_object_list_hbox,
            self.query_counter_selector_hbox,
        ]

        self.query_other_setting_view = Tab(
            childrens=[
                Box([
                    self.query_filter_time_start_hbox,
                    self.query_filter_time_end_hbox,
                    self.query_filter_time_hbox,
                ], layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 0px',
                    align_items='stretch',
                )),
                Box([
                    self.query_instance_name_selector_hbox,
                    self.query_filter_instance_name_hbox,
                    self.query_filter_instance_name_regex_hbox,
                ], layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 0px',
                    align_items='stretch',
                )),
                Box([
                    self.query_filter_min_value_hbox,
                    self.query_filter_max_value_hbox,
                ], layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 0px',
                    align_items='stretch',
                )),
            ],
            labels=["Filter Time", "Filter Instance", "Filter Value"])

        self.query_section_2 = [
            self.query_other_setting_view,
            widgets.HBox([
                self.query_make_table,
                self.query_make_plot,
            ], layout=form_item_layout),
            widgets.HBox([
                self.query_merge_results,
            ], layout=form_item_layout),
            self.status_query_hbox,
            self.query_display_counters_button_hbox,
        ]

        self.view_files_section = [
            self.view_files_workspace_id_hbox,
            widgets.HBox([
                self.view_files_refresh_workspace_ids_button_hbox,
                self.view_files_retrieve_button_hbox,
            ], layout=form_item_layout),
            self.view_files_filter_selector_hbox,
            self.view_files_cluster_name_hbox,
            self.view_files_node_names_hbox,
            self.view_files_gap_threshold_value_hbox,
            self.status_view_files_hbox,
            self.view_files_display_button_hbox,
        ]

        self.ingest_section_column = Box(self.ingest_section, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='475px',
        ))

        self.query_section_1_column = Box(self.query_section_1, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='475px',
        ))

        self.query_section_2_column = Box(self.query_section_2, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='500px',
        ))

        self.view_files_section_column = Box(self.view_files_section, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='500px',
        ))

        self.ingest_section_row = Box([self.ingest_section_column], layout=Layout(
            display='inline-flex',
            border='none',
            align_items='stretch',
            min_width='600px',
        ))

        self.query_section_row = Box([self.query_section_1_column, self.query_section_2_column], layout=Layout(
            display='flex',
            border='none',
            align_items='stretch',
            min_width='600px',
        ))

        self.view_files_section_row = Box([self.view_files_section_column], layout=Layout(
            display='inline-flex',
            border='none',
            align_items='stretch',
            min_width='600px',
        ))

        self.select_tab_view.observe(self.events.select_tab_view_on_change)
        self.query_object_list.observe(self.events.query_object_list_on_change)
        self.query_cluster_name.observe(
            self.events.query_cluster_name_on_change)
        self.query_node_names.observe(self.events.query_node_names_on_change)
        self.query_filter_time.observe(self.events.query_time_filter_on_change)
        self.notebook_url.observe(self.events.notebook_url_on_change)

        self.query_filter_instance_name.observe(
            self.events.query_instance_name_filter_on_change)
        self.query_filter_instance_name_regex.observe(
            self.events.query_instance_name_regex_filter_on_change)

        self.ingest_checkbox.observe(self.events.ingest_checkbox_on_change, names='value')

        self.ingest_button.on_click(self.events.ingest_button_on_click)
        self.query_refresh_workspace_ids_button.on_click(
            self.events.query_refresh_workspace_ids_button_on_click)
        self.query_retrieve_button.on_click(
            self.events.query_retrieve_button_on_click)
        self.query_display_counters_button.on_click(
            self.events.query_display_counters_button_on_click)

        self.view_files_refresh_workspace_ids_button.on_click(
            self.events.view_files_refresh_workspace_ids_button_on_click)
        self.view_files_retrieve_button.on_click(
            self.events.view_files_retrieve_button_on_click)
        self.view_files_display_button.on_click(
            self.events.view_files_display_button_on_click)

        self.status_ccma.value = "Enter required information"

        self.tab_form = Tab(
            childrens=[self.query_section_row,
                       self.view_files_section_row, self.ingest_section_row],
            labels=[
                "Query {0}".format(self.profile.workspace),
                "View {0}".format(self.profile.workspace),
                "Ingest into {0}".format(self.profile.workspace),
            ],
        )

        self.interactive_tab_view = interactive(self.display_tab_view)

        self.tab_view_vbox = widgets.VBox([
            widgets.HBox([
                widgets.Label("Select View:"), self.select_tab_view,
            ]),
            self.interactive_tab_view,
        ], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='100%',
        ))

        display(self.tab_form)
        display(self.tab_view_vbox)
        self.ingest_checkbox_label.layout.visibility="hidden"

    def build_plot(self, cm_table, counter_column_name):
        data = []

        for instance, group in cm_table.groupby('node_name'):
            data.append(go.Scatter(x=group['plot_timestamp'],
                                   y=group[counter_column_name],
                                   name=instance))

        layout = go.Layout(showlegend=True, title=counter_column_name)
        fig = go.Figure(data=data, layout=layout)
        return go.FigureWidget(fig)

    def display_tab_view(self):
        if self.select_tab_view.value == '':
            return

        index = self.tab_view_labels.index(self.select_tab_view.value)

        self.tab_view = widgets.HBox([
            self.tab_view_childrens[index],
        ], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 0px',
            align_items='stretch',
            width='100%',
        ))

        display(self.tab_view)

    def create_table_view(self, cm_table):
        if "plot_timestamp" in cm_table:
            cm_table = cm_table.drop(["plot_timestamp"], axis=1)

        # work around bug in beakerx 1.4.x. Remove after upgrade to 1.5.0
        for col in cm_table.columns:
            if cm_table[col].dtype.name == 'bool':
                cm_table[col] = cm_table[col].astype('str')

        beakerx_table = TableDisplay(cm_table)
        beakerx_table.setTimeZone("UTC")

        return beakerx_table

    def create_plot_view(self, cm_table, counter_column_name):
        fig = self.build_plot(cm_table, counter_column_name)

        return fig

In [ ]:
ASUP = ExplorerProfile(
    workspace="Case",
    workspace_plural="Cases",
    workspace_id="Case Id",
    workspace_id_plural="Case Ids",
    record_id="ASUP Id",
    record_id_plural="ASUP Ids",
)

form = ExplorerForm('asup', ASUP)
form.build_form()

In [ ]:
%%html
<script>
code_show=true;

function code_toggle() {
    if (code_show){
        $('div.input').hide();
    } else {
        $('div.input').show();
    }
    code_show = !code_show
} 
    
$( document ).ready(code_toggle);
</script>
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 200px
}
</style>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>